In [0]:
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from google.colab import drive
from google.colab import files
import pandas as pd
import numpy as np
import glob

# Make sure you have access to the images' folders
drive.mount('/content/drive/')

Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
cd drive/Team Drives

/content/drive/Team Drives


In [0]:
cd ECENproject2

/content/drive/Team Drives/ECENproject2


In [0]:
####The Convolutional Neural Network for stitched images####
#----------------------------------------------------------#

# Still requires tweaking. Also keep it simple for faster forward\backward propagations

# model = Sequential()
# model.add(Conv2D(16, (3, 3), input_shape= (640, 180, 1), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Conv2D(16, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Conv2D(16, (3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.3))
# model.add(Flatten())
# model.add(Dense(20, activation='relu'))
# model.add(Dense(2, activation='softmax'))

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape= (640, 180, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dense(200, activation='relu'))
model.add(Dense(2, activation='softmax'))

In [0]:
####       Running the CNN          ####
#--------------------------------------#

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 638, 178, 32)      320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 319, 89, 32)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 317, 87, 64)       18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 158, 43, 64)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 156, 41, 64)       36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 78, 20, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 74, 16, 64)        102464    
__________

In [0]:
nb_epochs = 100
batch_size = 15
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.05,
    zoom_range=0.05,
    horizontal_flip=True,
    validation_split=.05)

train_generator = train_datagen.flow_from_directory(
    'stitched_pic_with_far_objs',
    target_size=(640, 180),#width , height 
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode='binary',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    'stitched_pic_with_far_objs',
    target_size=(640, 180),
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode='binary',
    subset='validation') # set as validation data

model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = nb_epochs)

# model.save("MB_monoscopic_unseen_stitched_result.h5")

Found 2640 images belonging to 2 classes.
Found 138 images belonging to 2 classes.
Epoch 1/100
176/176 [==============================] - 41s 235ms/step - loss: 0.0077 - acc: 0.9973 - val_loss: 1.3664 - val_acc: 0.7556
Epoch 2/100
176/176 [==============================] - 41s 233ms/step - loss: 0.0122 - acc: 0.9973 - val_loss: 0.6215 - val_acc: 0.8211
Epoch 3/100
176/176 [==============================] - 41s 236ms/step - loss: 0.0048 - acc: 0.9985 - val_loss: 1.1919 - val_acc: 0.8699
Epoch 4/100
176/176 [==============================] - 41s 233ms/step - loss: 0.0153 - acc: 0.9962 - val_loss: 1.0890 - val_acc: 0.7886
Epoch 5/100
176/176 [==============================] - 41s 231ms/step - loss: 0.0026 - acc: 0.9992 - val_loss: 1.2431 - val_acc: 0.8293
Epoch 6/100
176/176 [==============================] - 41s 234ms/step - loss: 0.0198 - acc: 0.9955 - val_loss: 0.5263 - val_acc: 0.8211
Epoch 7/100
176/176 [==============================] - 40s 230ms/step - loss: 0.0064 - acc: 0.9985 - 

# GOOD MODELS